In [5]:
import pandas as pd

In [6]:
test = pd.read_csv('SF_crime/test.csv', index_col='Id')
test = test.rename(columns={'X': 'Longitude', "Y": "Latitude"})
test.Dates = pd.to_datetime(test.Dates)

results = pd.read_csv("predictions_XGB2.csv", index_col='Id')

In [10]:
def prob_to_column(df):
    most_likely = []
    second_most = []
    third_most = []
    df_t = df.T
    n=0
    for column in df_t:
        name1 = df_t[column].sort_values(ascending=False).to_frame().head(3).index.values[0]
        prob1 = df_t[column].sort_values(ascending=False)[0]
        name2 = df_t[column].sort_values(ascending=False).to_frame().head(3).index.values[1]
        prob2 = df_t[column].sort_values(ascending=False)[1]
        name3 = df_t[column].sort_values(ascending=False).to_frame().head(3).index.values[2]
        prob3 = df_t[column].sort_values(ascending=False)[2]
        Id = n
        most_likely.append((Id, name1, prob1))
        second_most.append((Id, name2, prob2))
        third_most.append((Id, name3, prob3))
        n+=1
    most_likely = pd.DataFrame(most_likely)
    second_most = pd.DataFrame(second_most)
    third_most = pd.DataFrame(third_most)
    most_likely = most_likely.rename(columns={0:'Id', 1:'most_likely', 2:'highest_probability'})
    second_most = second_most.rename(columns={0:'Id', 1:'second_most_likely', 2:'second_hightest_prob'})
    third_most = third_most.rename(columns={0:'Id', 1:'third_most_likely', 2:'third_highest_prob'})
    top_three = pd.merge(most_likely, second_most, on=most_likely.Id)
    top_three = pd.merge(top_three, third_most, on=third_most.Id)
    top_three = top_three.drop(['Id_y', 'Id'], axis=1)
    top_three = top_three.rename(columns={'Id_x':'Id'})
    return top_three

In [11]:
results_df = prob_to_column(results)

In [24]:
results_df.head()

,Id,most_likely,highest_probability,second_most_likely,second_hightest_prob,third_most_likely,third_highest_prob
0,0,OTHER OFFENSES,0.203127,VEHICLE THEFT,0.162556,VANDALISM,0.105344
1,1,OTHER OFFENSES,0.804285,NON-CRIMINAL,0.039011,VEHICLE THEFT,0.031694
2,2,LARCENY/THEFT,0.618779,NON-CRIMINAL,0.073270,VEHICLE THEFT,0.056663
3,3,OTHER OFFENSES,0.250367,LARCENY/THEFT,0.225559,VANDALISM,0.086718
4,4,OTHER OFFENSES,0.250367,LARCENY/THEFT,0.225559,VANDALISM,0.086718


In [30]:
for item in (results_df.third_most_likely.unique()):
    if item not in (results_df.second_hightest_prob.unique()):
        print(item)

KeyboardInterrupt: 

In [14]:
results_df.to_csv('/Users/MatthewBarnette/final_project_predictions//results_top_three.csv')

In [15]:
test_results = test.join(results_df)

In [16]:
test_results.to_csv('/Users/MatthewBarnette/final_project_predictions//test_results_top_three.csv')

In [17]:
testing = pd.read_csv('/Users/MatthewBarnette/final_project_predictions//test_results_top_three.csv')